<div style="width:100vw;margin-left:calc(50% - 50vw);"><table width="100%" cellspacing="10" cellpadding="14" bgcolor="#f9fafb" style="border:1px solid #d1d5db;border-radius:12px;"><tr><td width="33%" align="center" valign="middle" bgcolor="#ffffff" style="border-radius:10px;"><a href="https://github.com/rikkirch/Computer_Pattern_and_Vision_Recognition/blob/main/Assignment/Assignment_1/Rikesh_Budhathoki_A1.ipynb" target="_parent"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" width="28"></a><br><span style="color:#000000;font-family:Georgia,serif;font-size:20px;font-weight:700;">View on GitHub</span></td><td width="33%" align="center" valign="middle" bgcolor="#ffffff" style="border-radius:10px;"><a href="https://nbviewer.org/github/rikkirch/Computer_Pattern_and_Vision_Recognition/blob/main/Assignment/Assignment_1/Rikesh_Budhathoki_A1.ipynb" target="_parent"><img src="https://upload.wikimedia.org/wikipedia/commons/thumb/3/38/Jupyter_logo.svg/883px-Jupyter_logo.svg.png" width="28"></a><br><span style="color:#000000;font-family:Georgia,serif;font-size:20px;font-weight:700;">Open in nbviewer</span></td><td width="33%" align="center" valign="middle" bgcolor="#ffffff" style="border-radius:10px;"><a href="https://colab.research.google.com/github/rikkirch/Computer_Pattern_and_Vision_Recognition/blob/main/Assignment/Assignment_1/Rikesh_Budhathoki_A1.ipynb" target="_parent"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" width="28"></a><br><span style="color:#000000;font-family:Georgia,serif;font-size:20px;font-weight:700;">Run in Colab</span></td></tr></table></div>


### **Step 0: Mount Google Drive**

In Google Colab, the runtime environment is temporary, meaning any files uploaded directly to Colab are lost when the session ends. Since our calibration images are stored permanently in **Google Drive**, we need to mount Drive into Colab so that the notebook can access those files.

By mounting Google Drive:

* We can **read calibration images directly from Drive**
* We avoid re-uploading images every time the runtime restarts
* Our data remains **persistent and organized** across sessions

Once mounted, Google Drive behaves like a normal directory in the Colab file system, allowing us to load images using standard file paths.

This step must be done **before** accessing any images stored in Google Drive.

---

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## **Step 1: Load Calibration Images from Google Drive**

In this step, the calibration images stored in Google Drive are accessed and prepared for processing.

After mounting Google Drive, the images are loaded from the specified directory using file-pattern matching. This allows the notebook to automatically collect all calibration images without manually listing each filename.

The main goals of this step are:

* to **verify that Google Drive is correctly mounted**
* to **confirm that all calibration images are accessible**
* to **ensure the correct number of images is loaded before proceeding**

This step does **not** perform any computer vision operations yet.
It serves as a validation step to confirm that the dataset is correctly located and ready for further processing.

If the images cannot be loaded correctly at this stage, subsequent steps such as corner detection and camera calibration will fail.

---

### **Outcome of this step**

* A list of calibration image file paths is successfully created
* The total number of images is confirmed
* The dataset is ready for grayscale conversion and chessboard corner detection

---

### **Why this step is important**

Camera calibration relies on multiple images taken from different viewpoints. Ensuring that all calibration images are properly loaded at the beginning helps avoid silent errors later in the pipeline and guarantees reproducibility of the results.

---

In [ ]:
import cv2
import matplotlib.pyplot as plt
import os
import numpy as np
import glob
from tqdm import tqdm

In [ ]:
img_paths = sorted(glob.glob("/content/drive/MyDrive/CSC_422/Assignment_1/Datasets/*.jpeg"))
num_img = len(img_paths)
print(f"Number of images: {num_img}")
for path in img_paths:
    print(path)

Number of images: 12
/content/drive/MyDrive/CSC_422/Assignment_1/Datasets/IMG_1.jpeg
/content/drive/MyDrive/CSC_422/Assignment_1/Datasets/IMG_10.jpeg
/content/drive/MyDrive/CSC_422/Assignment_1/Datasets/IMG_11.jpeg
/content/drive/MyDrive/CSC_422/Assignment_1/Datasets/IMG_12.jpeg
/content/drive/MyDrive/CSC_422/Assignment_1/Datasets/IMG_2.jpeg
/content/drive/MyDrive/CSC_422/Assignment_1/Datasets/IMG_3.jpeg
/content/drive/MyDrive/CSC_422/Assignment_1/Datasets/IMG_4.jpeg
/content/drive/MyDrive/CSC_422/Assignment_1/Datasets/IMG_5.jpeg
/content/drive/MyDrive/CSC_422/Assignment_1/Datasets/IMG_6.jpeg
/content/drive/MyDrive/CSC_422/Assignment_1/Datasets/IMG_7.jpeg
/content/drive/MyDrive/CSC_422/Assignment_1/Datasets/IMG_8.jpeg
/content/drive/MyDrive/CSC_422/Assignment_1/Datasets/IMG_9.jpeg


## **Step 3: Detect Chessboard Inner Corners in Each Image**

In this step, we detect the **inner corner points** of the chessboard pattern in each calibration image. These inner corners correspond to the intersection points between adjacent black and white squares and are consistent across all images.

The purpose of this step is to establish a correspondence between:

* **Known 3D points in the real world** (defined by the chessboard geometry), and
* **Observed 2D points in the image** (pixel locations of the detected corners).

For each image, the chessboard is first converted to grayscale to simplify processing. Then, OpenCV’s chessboard corner detection algorithm is applied to locate the inner corners. If the expected number of corners is successfully detected, their pixel coordinates are stored and optionally visualized on the image for verification.

Accurate corner detection is crucial for reliable camera calibration. Images with motion blur, poor lighting, extreme viewing angles, or partial occlusion may fail to produce correct corner detections and should be excluded.

**Outcome of this step:**

* A list of 2D image points (pixel coordinates of inner corners) for each valid image
* A visual confirmation that the detected corners align correctly with the chessboard pattern

These detected image points are later used together with the known 3D world coordinates to compute the camera’s intrinsic and distortion parameters.

---

In [ ]:
# Define inner-corner pattern size, 9 - columns, 6 - rows
# I have 10 x 7 chessboard so that makes 9 x 6 inner corners
pattern_size = (9,6)
